In [ ]:
def getList(dict):
    """
    This function returns the list of keys in a dictionary
    """
    return dict.keys()


CLASSES = {'Agaricus' : 0,'Amanita' : 1,'Boletus' : 2,'Cortinarius' : 3,'Entoloma' : 4,'Hygrocybe' : 5,
    'Lactarius' : 6,'Russula' : 7,'Suillus' : 8
                }

def get_label(key ,class_mush = CLASSES):
    """
    You can get the labeled name of the mushroom by giving 
    the index or the index by giving the key
    """
    if type(key) == str:
        return class_mush[key]
    elif type(key) == int:
        return list(CLASSES.keys())[list(CLASSES.values()).index(key)]


def upload_dir_labels(directory,mapping =CLASSES, k = 300):
    """
    give the list of photos from directory and the labels.
    k = number of pictures I want to take from each FOLDER. 
    """
    label = []
    dir= []
    list_of_mushrooms = getList(mapping)
    for f in list_of_mushrooms:
        pictures_dir = directory + f + '/'
        pictures = [pics for pics in listdir(pictures_dir) if isfile(join(pictures_dir , pics))]
        for idx, picture in enumerate(pictures[:k]): # take the first 300 pictures from each category
            label.append(get_label(f))  
            dir.append(pictures_dir + picture) 
        
    return dir, label

def image_reshape(dir, batch, new_size = (224,224),k = 300, gray =False):
    """
    give the list of photos with the correct pixel size you want to 
    downsize. If you want Gray then write True.
    """
    items_resized = []
    for idx, dir in enumerate(dir[(batch-1)*k:batch * k]):     
        item = cv2.imread(dir)
        if gray == True:
            item_gray = cv2.cvtColor(item, cv2.COLOR_BGR2GRAY)
            item_resized_gray = resize(item_gray, new_size)
            items_resized.append(item_resized_gray)
        else:
            item_resized_colored = resize(item, new_size)
            items_resized.append(item_resized_colored)
    with open(directory + "downsized/mushrooms_resized_part_"+ str(batch), "wb") as fp:   #Pickling
        pickle.dump(items_resized, fp)
    return items_resized


def downsize_in_batches(directories):
    """
    This function downsizes the images in batches of 300pics.
    directories = list of the directories of the images.
    """
    batch = 1
    for idx, dir in enumerate(directories):

        if idx % 300 == 0:
            print(batch)
            image_reshape(directories,batch)
            batch += 1 


def unpickle(directory , mapping):
    """
    This functions takes the data from a directory that have been
    downsized in banches ang gives back the X,y as arrays.
    """
    
    table_mushrooms_downsized = []
    directories, labels = upload_dir_labels(directory,mapping, k=300)
    for batch in range(1,10): # 10 is the number of batches
        with open(directory + "downsized/mushrooms_resized_part_"+ str(batch), "rb") as fp:   # Unpickling
            li = pickle.load(fp)
            table_mushrooms_downsized.append(li)

    # flatten the nested list into a 1D list
    mushrooms_downsized = list(itertools.chain(*table_mushrooms_downsized))
    if len(mushrooms_downsized) == len(labels):

        #Check if the number of pictures and labels are the same.
        X = np.asarray(mushrooms_downsized)
        y = np.asarray(labels)

    return X, y

def isqrt(n):
    """
    This returns the largest integer x for which x * x does not exceed n. 
    If you want to check if the result is exactly the square root, simply 
    perform the multiplication to check if n is a perfect square.
    """
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x
def plot_mushroom_coll(x = 25):
    """
    We can see a collection of random mushrooms with the labels
    x is the number of mushrooms you want to see and it will give 
    the appropriate number of n * n array.
    """
    for i in range(x):
        ran = random.randint(0,len(y)-1)
        plt.subplot(int(np.sqrt(x)), int(np.sqrt(x)), i+1)
        plt.imshow(X[ran], cmap='Greys')
        plt.axis('off')
        plt.text(0, 0, y[ran]) # displays y-values in each subplot

def pickle_(directory, file):
    """
    This function pickles the data.
    """
    with open(directory , "wb") as fp:   #Pickling
        pickle.dump(file, fp)



def split_preprocessed_data(X ,y, test_size = 0.1, pp = False):
    """
    This function returns the train and validataion data 
    preprocessed if necessary: in a flattened 1_D form
    with normalized X and y per 255.
    Split images (75%/15%/10%) and save to temporary folders
    """
    if pp == True:
        x= X.reshape(X.shape[0],-1)
        x_norm = x.astype("float32") / 255 # Scale images to the [0, 1] range IF not scaled
        # one-hot-encode labels
        y_ohe = to_categorical(y)
        X_train, X_test, y_train, y_test = train_test_split(x_norm, y_ohe, test_size = test_size, random_state=42)
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = test_size * 1.7, random_state=42) 
    else:
        x_norm = X.astype("float32") 
        # one-hot-encode labels
        y_ohe = to_categorical(y)
        X_train, X_test, y_train, y_test = train_test_split(x_norm, y_ohe, test_size = test_size, random_state=42)
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = test_size * 1.7, random_state=42)  
    # file_name = 'data/Data/Mushrooms/'
    
    # pickle_(file_name + "train", X_train)
    # pickle_(file_name + "train", y_train)
    # pickle_(file_name + "test",  X_test)
    # pickle_(file_name + "test",  y_test)
    # pickle_(file_name + "valid", X_valid)
    # pickle_(file_name + "valid", y_valid)  
    return X_train, X_test, y_train, y_test, X_valid, y_valid
